# Bibliotecas e configurações iniciais

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# Importação dos Dados

Carregamento dos dados e análise dos tipos e características das colunas:

In [2]:
df = pd.read_csv('dados.csv')
df.head()

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
0,38.0,49.0,735612.0,735624.0,113.0,11.0,12.0,12652.0,93.0,130.0,1707.0,100.0,TypeOfSteel_A300,1
1,1252.0,1348.0,355940.0,356016.0,1812.0,119.0,135.0,196003.0,NaN,132.0,1687.0,80.0,TypeOfSteel_A300,1
2,193.0,210.0,612201.0,612252.0,588.0,18.0,51.0,62182.0,73.0,135.0,1353.0,290.0,TypeOfSteel_A400,1
3,1159.0,1170.0,32914.0,32926.0,106.0,11.0,12.0,12792.0,100.0,134.0,1353.0,185.0,TypeOfSteel_A400,1
4,366.0,392.0,228379.0,228429.0,612.0,46.0,52.0,71337.0,103.0,127.0,1687.0,200.0,TypeOfSteel_A400,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   min_x_defect          967 non-null    float64
 1   max_x_defect          967 non-null    float64
 2   min_y_defect          967 non-null    float64
 3   max_y_defect          967 non-null    float64
 4   area_pixels           967 non-null    float64
 5   slab_width            967 non-null    float64
 6   slab_length           967 non-null    float64
 7   sum_pixel_luminosity  967 non-null    float64
 8   min_pixel_luminosity  729 non-null    float64
 9   max_pixel_luminosity  967 non-null    float64
 10  conveyer_width        967 non-null    float64
 11  slab_thickness        967 non-null    float64
 12  type_of_steel         967 non-null    object 
 13  defect_type           967 non-null    int64  
dtypes: float64(12), int64(1), object(1)
memory usage: 105.9+ KB


In [4]:
(967-729)/967 # calculo do percentual de dados faltantes

0.24612202688728024

Observações:
- Somente uma coluna com valores nulos ("min_pixel_luminosity") - entretanto, a quantidade é significativa: aprox. 24.61%.
- Predominância de colunas do tipo float (12/14 colunas). Ainda vamos verificar, mais adiante, se essas colunas são realmente float (ou se não poderiam ser convertidas para inteiro).
- Colunas restantes: uma do tipo inteiro e outra do tipo objeto.

**É IMPORTANTE, DAQUI EM DIANTE, TER EM MENTE O DICIONÁRIO DOS DADOS:**

- min_x_defect – Coordenada x inicial do defeito 
- max_x_ defect – Coordenada x final do defeito 
- min_y_ defect – Coordenada y inicial do defeito 
- max_y_ defect – Coordenada y final do defeito 
- area_pixels – Total de pixels presentes na placa 
- slab_width – Largura da placa (eixo X) 
- slab_length – Comprimento da placa (eixo Y)  
- sum_pixel_luminosity – Soma da luminosidade dos pixels 
- min_pixel_luminosity – Mínima luminosidade dos pixels 
- max_pixel_luminosity – Máxima luminosidade dos pixels  
- conveyer_width – Largura da esteira (correia) transportadora (eixo X) 
- type_of_steel – Identifica a classe do aço: pode pertencer à classe A300 ou A400 
- defect_type – Tipo de defeito da classe. Pode ser do tipo 0 ou do tipo 1. 

# Verificação da Integridade dos Dados

Tratamento de tipos, nulos e *outliers*:

### Nulos

Como visto anteriormente, temos uma quantidade considerável de nulos na coluna 'min_pixel_luminosity' e precisamos tratá-los:

In [5]:
# visualizazr a distribuição dos dados da coluna com nulos ('min_pixel_luminosity')
px.histogram(df, x='min_pixel_luminosity')

In [6]:
px.box(df, x='min_pixel_luminosity', points='all')

Observação curiosa: não existem valores no intervalo 80-90, isso configura um espaço vazio peculiar no boxplot. Seria interessante conversar com um profissional para saber por que esses valores de mínima luminosidade dos pixels aparentam ser impossíveis. 


Ponderando opções para substituição dos nulos:
- Deletar as linhas com os valores nulos não é uma opção muito boa, pois estaríamos jogando fora aprox. 24.61% dos dados.
- Preencher com zeros também não é uma boa opção dada a distribuição dos valores observada no gráfico acima.
- Também não é uma ocasião muito interessante para bfill ou ffill (já que não se trata da evolução de valores no decorrer do tempo).
- Preencher com a média ou com a mediana aparenta ser a mais razoável das opções.

In [7]:
# Média e mediana da coluna
mean = df['min_pixel_luminosity'].mean()
median = df['min_pixel_luminosity'].median()

print("Média: ", mean)
print("Mediana: ", median)

Média:  95.77914951989025
Mediana:  101.0


In [8]:
'''
# Verificar erro em uma amostra para cada uma das abordagens

# amostra de 5% dos valores da coluna 'min_pixel_luminosity'
col_min_pixel_luminosity = df['min_pixel_luminosity'].sample(frac=0.05)
col_min_pixel_luminosity.dropna(inplace=True) # remover nulos

# Qual seria o erro se substituíssemos os valores pela média?
mean_error = abs(col_min_pixel_luminosity.sum() - len(col_min_pixel_luminosity)*mean)

# Qual seria o erro se substituíssemos os valores pela mediana?
median_error = abs(col_min_pixel_luminosity.sum() - len(col_min_pixel_luminosity)*median)

print("Erro da média: ", mean_error)
print("Erro da mediana: ", median_error)'''


'\n# Verificar erro em uma amostra para cada uma das abordagens\n\n# amostra de 5% dos valores da coluna \'min_pixel_luminosity\'\ncol_min_pixel_luminosity = df[\'min_pixel_luminosity\'].sample(frac=0.05)\ncol_min_pixel_luminosity.dropna(inplace=True) # remover nulos\n\n# Qual seria o erro se substituíssemos os valores pela média?\nmean_error = abs(col_min_pixel_luminosity.sum() - len(col_min_pixel_luminosity)*mean)\n\n# Qual seria o erro se substituíssemos os valores pela mediana?\nmedian_error = abs(col_min_pixel_luminosity.sum() - len(col_min_pixel_luminosity)*median)\n\nprint("Erro da média: ", mean_error)\nprint("Erro da mediana: ", median_error)'

Após observação dos dois gráficos e dos valores de média e mediana, a minha escolha é substituir pela mediana já que seu valor (101) está dentro do intervalo de maior ocorrência (100-104).

In [9]:
df['min_pixel_luminosity'].fillna(median, inplace=True) # substitui nulos pela mediana

In [10]:
df['min_pixel_luminosity'].describe() # estatísticas descritivas após substituição	

count    967.000000
mean      97.064116
std       24.970998
min        0.000000
25%       95.000000
50%      101.000000
75%      107.000000
max      203.000000
Name: min_pixel_luminosity, dtype: float64

### Tipos

Verificando se floats não poderiam ser inteiros:

In [11]:
# verifica se todos os valores das colunas float terminam em .0 (se são inteiros)
for col in df.columns:
  if df[col].dtype == 'float64':
    print(df[col].astype(str).str.endswith('.0').value_counts())

True    967
Name: min_x_defect, dtype: int64
True    967
Name: max_x_defect, dtype: int64
True    967
Name: min_y_defect, dtype: int64
True    967
Name: max_y_defect, dtype: int64
True    967
Name: area_pixels, dtype: int64
True    967
Name: slab_width, dtype: int64
True    967
Name: slab_length, dtype: int64
True    967
Name: sum_pixel_luminosity, dtype: int64
True    967
Name: min_pixel_luminosity, dtype: int64
True    967
Name: max_pixel_luminosity, dtype: int64
True    967
Name: conveyer_width, dtype: int64
True    967
Name: slab_thickness, dtype: int64


O código acusou que todos os valores em todas as colunas float terminam em ".0" sendo, na realidade, valores inteiros. 

In [12]:
# Convertendo colunas float para int:
for col in df.columns:
  if df[col].dtype == 'float64':
    df[col] = df[col].astype(int)

df.info() # visualizar alterações

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   min_x_defect          967 non-null    int32 
 1   max_x_defect          967 non-null    int32 
 2   min_y_defect          967 non-null    int32 
 3   max_y_defect          967 non-null    int32 
 4   area_pixels           967 non-null    int32 
 5   slab_width            967 non-null    int32 
 6   slab_length           967 non-null    int32 
 7   sum_pixel_luminosity  967 non-null    int32 
 8   min_pixel_luminosity  967 non-null    int32 
 9   max_pixel_luminosity  967 non-null    int32 
 10  conveyer_width        967 non-null    int32 
 11  slab_thickness        967 non-null    int32 
 12  type_of_steel         967 non-null    object
 13  defect_type           967 non-null    int64 
dtypes: int32(12), int64(1), object(1)
memory usage: 60.6+ KB


Verificando se existe alguma coluna preenchida totalmente pelo mesmo valor (de forma que sua informação fosse irrelevante na análise):


In [13]:
count=0
for col in df.columns:
  if df[col].nunique() == 1: # retorna número de valores únicos
    print(col)
    count+=1
if count == 0:
  print("Nenhuma coluna com apenas um valor único")

Nenhuma coluna com apenas um valor único


### *Outliers*

Visualizando outliers "acusados" pelo boxplot para cada coluna numérica:

In [14]:
# Boxplot das colunas numéricas
def plot_box(df, col):
  fig = px.box(df, x=col, points='all', title=col)
  fig.show()

for col in df.columns:
  if df[col].dtype == 'int32':
    plot_box(df, col)

In [15]:
#df.columns

Observação interessante: os dados indicam que existem padrões de espessura das placas ("slab_thickness")

As colunas a seguir, por enquanto, não precisam de tratamento de *outliers* por terem todos os seus valores dentro dos limites do boxplot:
- 'min_x_defect' (Coordenada x inicial do defeito) 
- 'max_x_defect' (Coordenada x final do defeito)
- 'conveyer_width' (Largura da esteira (correia) transportadora (eixo X))

O resto das colunas precisa ser analisada caso a caso, pois é muito importante ter cuidado para não eliminar um caso raro, mas perfeitamente possível dos dados. 

Nesse momento também seria interessante conversar com um profissional da área para saber se poderiam ser definidas algumas regras duras. Exemplo: "Existe alguma coordenada / região da placa que nunca dá defeito?" Se sim, criar regra.

OBS: Quando se é possível saber a precisão da capturação dos dados isso ajuda a inferir se temos uma maior ou uma menor probabilidade de *outliers*. Para esse desafio vamos confiar que as tecnologias da Arcelor são as melhores disponíveis e, dessa forma, a medição dos dados é bastante precisa.

As colunas a seguir são casos que julgo que todos os pontos fora da *lower fence* e da *upper fence* são casos possíveis (esse julgamento decorre da observação da ordem de grandeza no gráfico e também do significado das variáveis). Por isso, não serão tratados - pelo menos não por enquanto:
- 'min_pixel_luminosity' (Mínima luminosidade dos pixels)
- 'max_pixel_luminosity' (Máxima luminosidade dos pixels)
- 'slab_thickness' (Espessura da placa (eixo z))

As colunas a seguir possuem um ponto em particular extremamente distante da *upper fence* e dos outros pontos:
- 'area_pixels' (Total de pixels presentes na placa)
- 'slab_width' (Largura da placa (eixo X))
- 'slab_length' (Comprimento da placa (eixo Y))
- 'sum_pixel_luminosity' (Soma da luminosidade dos pixels)

Naturalmente, julgaríamos esses pontos como *outliers* logo de cara. Entretanto, dados os significados dessas variáveis, percebemos que é perfeitamente possível que sejam verdadeiras quando dadas em conjunto, ou seja, quando todas elas descrevem uma mesma placa - que seria uma placa de tamanho maior.

Vamos, em seguida, verificar se esses pontos pertencem de fato à mesma linha(placa) no nosso *dataframe*:


In [16]:
# Exibindo linhas de registros específicos 
# Estamos procurando uma única placa cujos valores são:
# area_pixels = 37334, slab_width = 1275, slab_lenght = 903, sum_pixel_luminosity = 3918219 (extraídos dos plots)

df[df['area_pixels'] == 37334] 

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
650,15,149,2278497,2278909,37334,908,903,3918209,19,134,1387,40,TypeOfSteel_A400,0


Somente o valor para "slab_width" que não deu "*match*". Mas, é possível observar que se trata de uma placa maior que torna todos esses valores mais altos possíveis para os dados.

Agora vamos analisar a linha/placa para o valor de slab_width == 1275:

In [17]:
df[df['slab_width'] == 1275]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
466,59,1245,4712084,4712115,15775,1275,276,1689009,61,135,1354,100,TypeOfSteel_A300,0


Trata-se de outra placa grande, mesmo sendo menor que a anterior em area, que também torna possíveis os valores.

Vamos analisar um último ponto desses boxplots (o correto seria analisar todos, mas eu **ainda** não tenho conhecimento para fazer isso de forma automatizada e temos pouco tempo nesse desafio):

In [18]:
df[df['area_pixels'] == 19818]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
719,772,1006,4313950,4314267,19818,905,583,1966409,46,135,1358,200,TypeOfSteel_A300,0


Também configura dados possíveis.

Então também não temos *outliers* para essas colunas? Calma lá, que a vida não é um morango! 

Veja que temos a informação 'conveyer_width' que é a largura da esteira (correia) transportadora (eixo X). Assim, também precisamos verificar se essas placas grandes por acaso não excedem o tamanho da esteira - caso afirmativo, esses dados são *outliers* multivariados.

O conceito de outlier multivariado é exatamente quando você tem dados que, separadamente são possíveis, mas juntos não são. Nada nos impede de ter uma placa grandona de aço, mas se essa placa não cabe na esteira transportadora é porque algo foi medido errado!




In [19]:
# verifica se há linhas em que a placa não cabe na esteira:
count=0

for i in range(len(df)):
  if i in df.index: # necessario pois algumas linhas foram removidas 
    if df['slab_width'][i] > df['conveyer_width'][i] or df['slab_length'][i] > df['conveyer_width'][i]:
      print(df.iloc[i])
      count+=1

if count == 0:
  print("Nenhuma placa com dimensões maiores que a esteira")

Nenhuma placa com dimensões maiores que a esteira


Agora sim podemos afirmar que nossos valores são perfeitamente possíveis! :)

Seguindo a mesma lógica de análise temos as colunas: 
- "min_y_ defect" (Coordenada y inicial do defeito)
- "max_y_ defect" (Coordenada y final do defeito)
- "min_x_ defect" (Coordenada x inicial do defeito)
- "max_x_ defect" (Coordenada x final do defeito)

Não podemos ter uma coordenada de defeito em um ponto que esteja fora da placa... Portanto vamos verificar se as coordenadas estão dentro das medidas para as placas:

In [20]:
count=0

for i in range(len(df)):
  if i in df.index: # necessario pois algumas linhas foram removidas 
    if df['min_y_defect'][i] > df['slab_width'][i] & df['min_y_defect'][i] > df['slab_length'][i] & df['min_x_defect'][i] > df['slab_width'][i] & df['min_x_defect'][i] > df['slab_length'][i]:
      print(df.iloc[i])
      count+=1

if count == 0:
  print("Nenhum ponto de defeito fora da placa")

Nenhum ponto de defeito fora da placa


Repare que não possuímos as unidades de medida dessas informações, não sabemos se o tamanho da placa está em centimetros, metros ou etc. E nem sabemos quantos centimetros ou metros correspondem uma unidade de coordenada. O código não acusou nenhum *outlier* da forma que está, mas seria super importante, em um caso real, obter essas unidades de medidas para afirmar isso com certeza.

Outra busca por outlier multivariado pode ser feita verificando se a mínima luminosidade dos pixels é maior do que a máxima luminosidade dos pixels para uma mesma placa - situação impossível no mundo real:

In [21]:
# Verifica se há placas em que min_pixel_luminosity é maior que max_pixel_luminosity
df[df['min_pixel_luminosity'] > df['max_pixel_luminosity']]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
901,1662,1670,2729960,2729972,55,11,12,4727,101,93,1690,70,TypeOfSteel_A300,0


Há uma placa em que isso ocorre, sendo, portanto, um outlier multivariado - vamos eliminá-lo:

In [22]:
df.drop(df[df['min_pixel_luminosity'] > df['max_pixel_luminosity']].index, inplace=True)

Assim, finalizamos nossa busca por *outliers* nas colunas numéricas, a coluna categórica vai ser analisada mais adiante.

# Análise Exploratória

Local onde formulamos diversas perguntas buscando tirar informações relevantes sobre os dados.

**Quantas classes temos e como é a distribuição de aço entre elas?**

In [23]:
# Análise da coluna categórica 'type_of_steel'
px.histogram(data_frame=df, x='type_of_steel', title='Distribuição de aço por classe')


Temos quatro classes e há uma "superconcentração" dos dados em duas delas.

Hipótese: pode ser que "TypeOfSteel_A300" e "TypeOfStel_A300" sejam a mesma classe com apenas um erro de ortografia enquanto "TypeOfSteel_????" seja também um erro de preenchimento dos dados. Se essa hipótese for válida, na prática teríamos apenas duas classes de aço: A300 e A400. 

Em um projeto seria muito importante recorrer à um responsável para tirar essa dúvida e consertar esses erros sem deduzir nada sozinho.

Aqui, entretanto, como se trata de um desafio e não podemos confirmar, vou assumir a hipótese como verdadeira e tratar.

In [24]:
# Corrigindo erro de ortografia 
df['type_of_steel'] = df['type_of_steel'].str.replace('TypeOfStel_A300', 'TypeOfSteel_A300')

# Excluindo dados de "TypeOfSteel_????" (pois são poucos)
df.drop(df[df['type_of_steel'] == 'TypeOfSteel_????'].index, inplace=True)

# Resultado
px.histogram(data_frame=df, x='type_of_steel', title='Distribuição de aço por classe')

**Como está a distribuição dos defeitos nos dados?**

In [25]:
# Visualização da coluna alvo ('defect_type')
df['defect_type'].value_counts()

0    600
1    357
Name: defect_type, dtype: int64

In [26]:
(606-361)/606

0.4042904290429043

In [27]:
px.histogram(data_frame=df, x='defect_type', title='Distribuição de defeitos', color='defect_type')

Temos uma distribuição de 40.43% de peças com defeito "1" *vs* 59.57% de peças com defeito "0' - distribuição razoavelmente equilibrada

**Como está a distribuição de defeitos dentro das classes existentes?**

In [28]:
# Distribuição de defeitos em cada classe de aço
px.histogram(df, x='type_of_steel', color='defect_type', title='Distribuição de defeitos por classe')

O aço A300 é mais equilibrado entre os tipos de defeitos enquanto o A400 possui uma predominância significativa do defeito tipo 0

**Como está a distribuição de defeitos nas colunas numéricas?**

In [29]:
# São muitas colunas, vamos plotar aos poucos dessa vez

# Plota histograma por defect_type passando colunas específicas 
def plot_hist_defect(df, columns):
    for col in columns:
        fig = px.histogram(df, x=col, color='defect_type', title=col)
        fig.show()

plot_hist_defect(df, ['min_x_defect', 'max_x_defect', 'min_y_defect', 'max_y_defect'])

Para situações de coordenada muito baixa em x e y ou muito alta em y, predomina o defeito tipo 0

In [30]:
#df.columns

In [31]:
plot_hist_defect(df, ['area_pixels', 'slab_width', 'slab_length', 'sum_pixel_luminosity'])

Para valores mais elevados (extremo superior) em todas essas colunas, predomina o defeito tipo 0

In [32]:
plot_hist_defect(df, ['min_pixel_luminosity', 'max_pixel_luminosity'])

Para ambas as extremidades (inferior e superior), predomina o defeito tipo 0



In [33]:
plot_hist_defect(df, ['slab_thickness', 'conveyer_width']) # descomentar para visualizar

Por fim, não foram extraídas observações relevantes das colunas 'slab_thickness' e 'conveyer_width', as distribuições são equilibradas - Código comentado para ocupar menos espaço. ERRADO: tem concentração de dados sim

**Como está a distribuição de classes nas colunas numéricas?**

In [3403]:
# Plota histograma por type_of_steel passando colunas específicas 
def plot_hist_type(df, columns):
    for col in columns:
        fig = px.histogram(df, x=col, color='type_of_steel', title=col)
        fig.show()

plot_hist_type(df, ['min_x_defect', 'max_x_defect', 'min_y_defect', 'max_y_defect'])

A distribuição dos tipos de aço nas coordenadas dos defeitos parece equilibrada para o eixo x, já o eixo y possui uma leve predominância do aço A400 em valores mais elevados.

In [3404]:
#plot_hist_type(df, ['area_pixels', 'slab_width', 'slab_length', 'sum_pixel_luminosity']) # descomentar para visualizar

"Distribuições equilibradas para as colunas 'area_pixels', 'slab_width', 'slab_length' e 'sum_pixel_luminosity' - Código comentado para ocupar menos espaço.

In [3405]:
plot_hist_type(df, ['min_pixel_luminosity', 'max_pixel_luminosity'])

Para a mínima luminosidade dos pixels:
- Predominância do aço A300 para a janela de valores de 35 a 79 
- Somente aço A400 acima de 135

Para a máxima luminosidade dos pixels:
- Predominância do aço A300 na janela de valores de 35 a 114

In [3406]:
plot_hist_type(df, ['slab_thickness'])

Apenas o aço A400 compõe os dados de placas com a espessura mais fina (40)

In [3407]:
plot_hist_type(df, ['conveyer_width'])

O aço A300 é majoritariamente transportado por esteiras maiores, enquanto o A400, por esteiras menores. 


**Como fica o mapa de calor dos defeitos?**
UMAP? [VOLTAR AQUI]

# Encoding

É necessário aplicar o *encoding* na nossa coluna categórica (type_of_steel) para que o modelo lide com ela.

In [3408]:
from sklearn.preprocessing import OrdinalEncoder

df_enc = df.copy()
ord_enc = OrdinalEncoder(dtype=int) # instanciando o objeto

ord_enc.fit(df[['type_of_steel']]) # ajuste
df_enc['type_of_steel'] = ord_enc.transform(df[['type_of_steel']]) # transformação

df_enc.head()

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
0,38,49,735612,735624,113,11,12,12652,93,130,1707,100,0,1
1,1252,1348,355940,356016,1812,119,135,196003,101,132,1687,80,0,1
2,193,210,612201,612252,588,18,51,62182,73,135,1353,290,1,1
3,1159,1170,32914,32926,106,11,12,12792,100,134,1353,185,1,1
4,366,392,228379,228429,612,46,52,71337,103,127,1687,200,1,1


Agora temos na coluna de tipo de aço o valor 0 representando o aço A300 e o valor 1 representando o aço A400

In [3409]:
df_enc.info() # visualizar alteração - tudo tipo inteiro

<class 'pandas.core.frame.DataFrame'>
Int64Index: 957 entries, 0 to 966
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   min_x_defect          957 non-null    int32
 1   max_x_defect          957 non-null    int32
 2   min_y_defect          957 non-null    int32
 3   max_y_defect          957 non-null    int32
 4   area_pixels           957 non-null    int32
 5   slab_width            957 non-null    int32
 6   slab_length           957 non-null    int32
 7   sum_pixel_luminosity  957 non-null    int32
 8   min_pixel_luminosity  957 non-null    int32
 9   max_pixel_luminosity  957 non-null    int32
 10  conveyer_width        957 non-null    int32
 11  slab_thickness        957 non-null    int32
 12  type_of_steel         957 non-null    int32
 13  defect_type           957 non-null    int64
dtypes: int32(13), int64(1)
memory usage: 63.6 KB


# XGBOOST

A escolha do modelo Extreme Gradient Boosting se deve a:
- regularização embutida para evitar o *overfitting*
- ser um algoritmo de árvore conhecido por seu desempenho superior
- ser um problema de classificação
- lida bem com dados desbalanceados

OBS: Modelos de árvore não precisam ser normalizados! 

### Separação em treino e teste usando validação cruzada

In [3410]:
# Separar coluna alvo defect_type
X = df_enc.drop('defect_type', axis=1)
y = df_enc['defect_type'].to_frame() # transforma em dataframe 

In [3411]:
X.head()

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel
0,38,49,735612,735624,113,11,12,12652,93,130,1707,100,0
1,1252,1348,355940,356016,1812,119,135,196003,101,132,1687,80,0
2,193,210,612201,612252,588,18,51,62182,73,135,1353,290,1
3,1159,1170,32914,32926,106,11,12,12792,100,134,1353,185,1
4,366,392,228379,228429,612,46,52,71337,103,127,1687,200,1


In [3412]:
y.head()

,defect_type
0,1
1,1
2,1
3,1
4,1


In [3413]:
from sklearn.model_selection import StratifiedKFold # O StratifiedKFold preserva a proporção de classes em cada fold

N_folds = 5 # número de grupos

# Criação do Splitter
splitter = StratifiedKFold (
    n_splits=N_folds,
    shuffle=True # embaralha os dados antes de dividir
    )

# Listas para armazenar os dados de cada fold
X_train_fold = []
y_train_fold = []
X_test_fold = []
y_test_fold = []

# Divisão dos dados 
for index_train, index_test in splitter.split(X, y):
  X_train = X.iloc[index_train]
  y_train = y.iloc[index_train]
  X_test = X.iloc[index_test]
  y_test =  y.iloc[index_test]

  # adicionar na lista
  X_train_fold.append(X_train)
  y_train_fold.append(y_train)
  X_test_fold.append(X_test)
  y_test_fold.append(y_test)

### Treino do modelo

In [3414]:
# Apesar de termos como principal a métrica do f1, vamos calcular outras métricas também para entender melhor o modelo como um todo:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from xgboost import XGBClassifier

# Esse é o nosso dicionário que vai conter as métricas do treino
train_metrics = {
    'accuracy': [],
    'recall': [],
    'precision': [],
    'f1': []
}

# Esse é o nosso dicionário que vai conter as métricas do teste
test_metrics = {
    'accuracy': [],
    'recall': [],
    'precision': [],
    'f1': []
}

# Listas para guardar as previsões feitas pro grupo de treino e teste
y_hat_train_fold = []
y_hat_test_fold = []
y_hat_test_proba_fold = []

In [3415]:
# Verificar hiperparâmetros do XGBoost
XGBClassifier().get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [3416]:
# Definir hiperparâmetros do modelo 

# Minha otimização manual:
hyper_dict = {
  'objective':'binary:hinge', # define que o tipo do problema é uma classificação binária
  'learning_rate': 0.05, # Taxa de aprendizado: quanto menor essa taxa, mais arvores voce vai precisar para chegar no seu aprendizado
  'min_child_weight': 60, # Soma mínima do peso da instância necessária em um filho - similar ao min_samples_leaf dos outros algoritmos de árvore.
                          # Apenas lembrando que 'min_samples_leaf' é o número mínimo de amostras numa folha (quando esse numero é 1 vc cria um overfiting)
  'max_depth': 5, # Profundidade máxima da árvore (quantas escolhas em sequencia eu posso fazer)
  'subsample': 0.85, # Dita a fração de linhas do banco analisadas a cada boost. O padrão é 1, indicando 100% das linhas.
}

# Otimização do GridSearchCV:
'''hyper_dict = {
  'objective':'binary:hinge', # define que o tipo do problema é uma classificação binária
  'learning_rate': 0.03, # Taxa de aprendizado: quanto menor essa taxa, mais arvores voce vai precisar para chegar no seu aprendizado
  'min_child_weight': 10, # Soma mínima do peso da instância necessária em um filho - similar ao min_samples_leaf dos outros algoritmos de árvore.
                          # Apenas lembrando que 'min_samples_leaf' é o número mínimo de amostras numa folha (quando esse numero é 1 vc cria um overfiting)
  'max_depth': 4, # Profundidade máxima da árvore (quantas escolhas em sequencia eu posso fazer)
  #'subsample': 0.8, # Dita a fração de linhas do banco analisadas a cada boost. O padrão é 1, indicando 100% das linhas.
  #'colsample_bytree': 0.8, # Dita a fração de colunas do banco analisadas a cada boost. O padrão é 1, indicando 100% das colunas.
}'''

"hyper_dict = {\n  'objective':'binary:hinge', # define que o tipo do problema é uma classificação binária\n  'learning_rate': 0.03, # Taxa de aprendizado: quanto menor essa taxa, mais arvores voce vai precisar para chegar no seu aprendizado\n  'min_child_weight': 10, # Soma mínima do peso da instância necessária em um filho - similar ao min_samples_leaf dos outros algoritmos de árvore.\n                          # Apenas lembrando que 'min_samples_leaf' é o número mínimo de amostras numa folha (quando esse numero é 1 vc cria um overfiting)\n  'max_depth': 4, # Profundidade máxima da árvore (quantas escolhas em sequencia eu posso fazer)\n  #'subsample': 0.8, # Dita a fração de linhas do banco analisadas a cada boost. O padrão é 1, indicando 100% das linhas.\n  #'colsample_bytree': 0.8, # Dita a fração de colunas do banco analisadas a cada boost. O padrão é 1, indicando 100% das colunas.\n}"

Alguns resultados dos testes manuais foram colocados em um arquivo excel 'Testes_realizados_hiperparametros' que estou enviando juntamente com o desafio.

O GridSearchCV (otimizador de parâmetros) pareceu se importar apenas com a obtenção das melhores métricas, não ponderando se está overfitando ou não... Dessa forma, acabei ficando com minha otimização manual.

OBS: GridSearchCV implementado mais abaixo.

In [3417]:
# Executar para cada fold
for fold in range(N_folds):

  # Recuperando os dados desse fold específico
  X_train = X_train_fold[fold]
  X_test = X_test_fold[fold]
  y_train = y_train_fold[fold]
  y_test = y_test_fold[fold]

  # Treinar o modelo
  model = XGBClassifier(**hyper_dict)
  model.fit(X_train, y_train.iloc[:,0])

  # Realizar as previsões
  y_hat_train = model.predict(X_train)
  y_hat_test = model.predict(X_test)
  y_hat_proba_test = model.predict_proba(X_test)

  # Salvar previsões para o conjunto de treino
  y_hat_train_fold.append(y_hat_train)
  y_hat_test_fold.append(y_hat_test)
  y_hat_test_proba_fold.append(y_hat_proba_test)

  # Calcular métricas do treino
  acc = accuracy_score(y_train, y_hat_train)
  train_metrics['accuracy'].append(acc)

  rec = recall_score(y_train, y_hat_train, average='weighted') # 'weighted' dá mais peso para as classes com menos amostras
  train_metrics['recall'].append(rec)

  precision = precision_score(y_train, y_hat_train, average='weighted')
  train_metrics['precision'].append(precision)

  f1 = f1_score(y_train, y_hat_train, average='weighted')
  train_metrics['f1'].append(f1)

  # Calcular métricas do teste
  acc = accuracy_score(y_test, y_hat_test)
  test_metrics['accuracy'].append(acc)

  rec = recall_score(y_test, y_hat_test, average='weighted')
  test_metrics['recall'].append(rec)

  precision = precision_score(y_test, y_hat_test, average='weighted')
  test_metrics['precision'].append(precision)

  f1 = f1_score(y_test, y_hat_test, average='weighted')
  test_metrics['f1'].append(f1)

In [3418]:
#train_metrics # descomentar para visualizar

In [3419]:
#test_metrics # descomentar para visualizar

### Otimização de hiperparâmetros com GridSearchCV
Espaço que utilizei o otimizador e deixei o código comentado após encontrar os melhores parâmetros indicados por ele.

In [3420]:
'''from sklearn.model_selection import GridSearchCV

# Definir hiperparâmetros do modelo
hiper_dict = {
    'objective':['binary:hinge'],
    'learning_rate': [0.03, 0.04, 0.05],
    'min_child_weight': [10, 20, 30, 40],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.7, 0.8, 0.9,],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Instanciar o modelo
model = XGBClassifier()

# Instanciar o GridSearch
grid_search = GridSearchCV(
    estimator=model,
    param_grid=hiper_dict,
    scoring='f1_weighted', # métrica que será usada para avaliar o modelo
    cv=5, # número de folds
    #verbose=1, # mostra o progresso do treinamento
    n_jobs=-1 # usa todos os núcleos do processador
)

# Treinar o modelo
grid_search.fit(X, y.iloc[:,0])

# Melhores parâmetros
grid_search.best_params_'''

"from sklearn.model_selection import GridSearchCV\n\n# Definir hiperparâmetros do modelo\nhiper_dict = {\n    'objective':['binary:hinge'],\n    'learning_rate': [0.03, 0.04, 0.05],\n    'min_child_weight': [10, 20, 30, 40],\n    'max_depth': [3, 4, 5, 6],\n    'subsample': [0.7, 0.8, 0.9,],\n    'colsample_bytree': [0.7, 0.8, 0.9]\n}\n\n# Instanciar o modelo\nmodel = XGBClassifier()\n\n# Instanciar o GridSearch\ngrid_search = GridSearchCV(\n    estimator=model,\n    param_grid=hiper_dict,\n    scoring='f1_weighted', # métrica que será usada para avaliar o modelo\n    cv=5, # número de folds\n    #verbose=1, # mostra o progresso do treinamento\n    n_jobs=-1 # usa todos os núcleos do processador\n)\n\n# Treinar o modelo\ngrid_search.fit(X, y.iloc[:,0])\n\n# Melhores parâmetros\ngrid_search.best_params_"

# Avaliação das métricas

### Médias das métricas

In [3421]:
# Média das métricas de treino
print("Média das métricas de TREINO:")
for key in train_metrics.keys():
  print(key, "=", sum(train_metrics[key])/len(train_metrics[key]))

Média das métricas de TREINO:
accuracy = 0.8079984300073381
recall = 0.8079984300073381
precision = 0.8068807917457074
f1 = 0.8072011257011168


In [3422]:
# Média das métricas de teste
print("Média das métricas de TESTE:")
for key in test_metrics.keys():
  print(key, "=", sum(test_metrics[key])/len(test_metrics[key]))

Média das métricas de TESTE:
accuracy = 0.7512707242582897
recall = 0.7512707242582897
precision = 0.7519351879014671
f1 = 0.7503333996804581


### Matriz de Confusão

Para a matriz de confusão precisamos reunir os dados que foram separados em cinco grupos de volta em um único grupo

In [3423]:
# Converter de 5 listas de y's para uma única lista com todos os valores
y_test_list = [] # lista com os valores reais
for y_fold in y_test_fold: # pega os valores de cada fold
  y_test_list.extend(y_fold.iloc[:,0])

y_hat_test_list = [] # lista com os valores previstos
for lista_fold in y_hat_test_fold: # pega os valores de cada fold
  y_hat_test_list.extend(lista_fold)

y_test_hat_proba_list = [] # lista com as probabilidades
for lista_fold in y_hat_test_proba_fold: # pega as probabilidades de cada fold
  y_test_hat_proba_list.extend(lista_fold)

In [3424]:
import sklearn

confusion_matrix = pd.DataFrame(
    sklearn.metrics.confusion_matrix(y_test_list, y_hat_test_list),
    index=['real_0', 'real_1'],
    columns=['pred_0', 'pred_1'],
)

display(confusion_matrix.style.background_gradient(axis=None))

# armazena os valores da matriz de confusão para usar depois
cm = confusion_matrix.values # obtida do ultimo fold

,pred_0,pred_1
real_0,484,116
real_1,122,235


# Feature importance (importância das variáveis)

Medida de quão significativas ou influentes são as variáveis (*features*) utilizadas no modelo para realizar previsões. Lembrando que essa importância não necessariamente resume a realidade e sim o cenário dos dados fornecidos ao modelo.

In [3425]:
model.feature_importances_ # pega o ultimo que saiu do loop (temos 5, no caso o quinto)

array([0.04387803, 0.04772193, 0.08200847, 0.        , 0.04870669,
       0.06108857, 0.05266043, 0.05397275, 0.05665785, 0.0639894 ,
       0.07356843, 0.12743773, 0.28830975], dtype=float32)

In [3426]:
df_importance = pd.DataFrame( # cria um dataframe
    data=model.feature_importances_.reshape(-1,1), # transforma o que era uma linha em uma tabela
    index=X_train_fold[0].columns, # nome das colunas viram as linhas
    columns=['Features'] 
)
df_importance = df_importance.sort_values(by='Features', ascending=False) # ordena do maior para o menor
df_importance

,Features
type_of_steel,0.288310
slab_thickness,0.127438
min_y_defect,0.082008
conveyer_width,0.073568
max_pixel_luminosity,0.063989
slab_width,0.061089
min_pixel_luminosity,0.056658
sum_pixel_luminosity,0.053973
slab_length,0.052660
area_pixels,0.048707


A variável de maior importância na decisão (disparadamente) do modelo é o tipo do aço, seguido pela espessura da placa e pela coordenada de início do defeito no eixo Y, configurando o top 3.

In [3427]:
px.bar(df_importance, title='Importância de cada variável no modelo', orientation='h') # visualização em barra horizontal

Lembrando que as informações aqui foram retiradas do último grupo (fold) da validação cruzada.

# Perguntas

**Questão 1:** O modelo construído será utilizado para reduzir o erro na hora da classificação do defeito. Sabe-se que:  
- Cada acerto do modelo significa um custo de R$ 500,00 para a recuperação da placa;   
- Identificar o defeito 0 incorretamente como defeito 1 gera um custo de R$ 500,00 para recuperação da placa 
mais um custo de R$ 3500,00 por conta do custo logístico de ter enviado a placa para o tratamento incorreto; 
- Identificar o defeito 1 incorretamente como defeito 0 gera um custo de R$ 500,00 para a recuperação da placa. 
Além disso, há também um custo de R$ 6213,00 por conta do erro logístico de ter enviado a placa para o 
tratamento incorreto, sabendo que o tratamento do defeito 1 ocorre numa etapa do processo anterior ao do 
defeito 0. 

Em posse dessas informações, qual seria o custo total do processo com o uso em produção do modelo 
desenvolvido? Deixe bem claro todo o passo a passo utilizado para a obtenção do resultado. 

In [3428]:
# Custos fornecidos 
v0 = 500 # verdadeiro zero
v1 = 500 # verdadeiro um
f1 = 4000 # falso um: defeito 0 identificado como 1
f0 = 6713 # falso zero: defeito 1 identificado como 0

# Calcula o custo total
custo_modelo = cm[0,0]*v0 + cm[0,1]*f1 + cm[1,0]*f0 + cm[1,1]*v1

# Imprime em reais
print("Custo total: R${:,.2f}".format(custo_modelo)) 

Custo total: R$1,642,486.00


**Questão 2:** Sem uma ferramenta mais tecnológica ao seu dispor, atualmente a distinção dos dois defeitos é feita de 
forma manual por um especialista. Para esse conjunto de dados específico, ele obteve os seguintes resultados: 
- 350 placas com defeito tipo 0 foram identificadas corretamente; 
- 256 placas com defeito tipo 0 foram identificadas como defeito tipo 1; 
- 161 placas com defeito tipo 1 foram identificadas corretamente; 
- 200 placas com defeito tipo 1 foram identificadas como defeito tipo 0.  

Conhecendo o custo de cada tipo de erro (conforme questão 1), qual seria a economia que a utilização do seu 
modelo traria para o processo? Deixe bem claro todo o passo a passo utilizado para a obtenção do resultado.

In [3429]:
# Custos para os resultados do especialista
custo_especialista = 350*v0 + 161*v1 + 256*f1 + 200*f0

# Calcula a economia
economia = custo_especialista - custo_modelo

# Imprime os resultados
print("Custo especialista: R${:,.2f}".format(custo_especialista)) # imprime em reais
print("Economia: R${:,.2f}".format(economia)) # imprime em reais

Custo especialista: R$2,622,100.00
Economia: R$979,614.00


# Conclusões gerais

- Vários insights foram fornecidos durante as análises ao longo do código.
- A implementação do modelo traz uma economia relevante.
- Aqui no desafio levamos em consideração a métrica f1-score como principal (que é o equilíbrio entre *recall* e *precision*). Entretanto, sabendo que um defeito 0 identificado como 1 gera um custo maior, talvez uma melhoria futura pudesse ser dar mais relevância à métrica *recall*.
- Algumas limitações se deveram à não ser possível comunicar para se obter algumas informações faltantes como, por exemplo, as unidades de medida de cada variável fornecida. 